In [1]:
# Primer:
# =======
# The Interrupt Descriptor Table is a fundamental system datastructure
# that interface with the hardware. It is a table of 256 entries, each
# entry is 16 bytes long. Each entry contains a pointer to a function that
# is called when the interrupt is triggered. The function is called with
# the exception frame, which contains the state of the CPU when the
# interrupt was triggered.
# =======

In [2]:
import ctypes

import os
# Add path to LD_LIBRARY_PATH

symbios_path = "../../"
os.environ["LD_LIBRARY_PATH"] += ":" + symbios_path + "Symlib/dynam_build"

symbios_path = "../../"
# The Symbiote Library module allows us to read and write the IDT
sym_mod = ctypes.cdll.LoadLibrary(symbios_path + "Symlib/dynam_build/libSym.so")

In [3]:
# Print the IDTR register, which provides a base and limit
sym_mod.sym_print_idtr()

# The base is the virtual address of the IDT, the limit is the size of the IDT
# Limit is 0xfff 256 * 16 - 1 = 4095, so it ranges from base[bytes 0 to 4095], or 
# 256 possible entries, each 16 bytes long

idtr limit: 0xfff 
idtr base : 0xffffc9000053f000 


32

In [4]:
import subprocess
def run_cmd(cmd):
    ret = subprocess.run(cmd.split())
    if ret.returncode != 0:
        print("error: ", ret.returncode)

In [5]:
# A tool that uses the symlib to operate on the IDT
run_cmd(symbios_path + "Tools/bin/idt_tool -h")

./idt_tool:
options:
	-a <addr>: address of idt, current loaded assumed if not provided
	c:         copy idt return ptr to copy on kern pg
	g:         get current idtr
	h:         print this help msg
	i:         install idt (swing idtr)
	m <ist_enable|ist_disable|addr:0xaddr>: modify idt entry
	p:         print
	v <dec#>:  vector number for print / modify
	z <df|tf>: which mitigation to copy to kern page

examples:
	taskset -c 1 ./idt_tool -g
	taskset -c 1 ./idt_tool -c
	./idt_tool -z tf
	./idt_tool -a ffffc90000986000 -m addr:0xffffc9000098d000 -v 14
	taskset -c 1 ./idt_tool -a ffffc90000986000 -i

df mitigation workflow:
	taskset -c 0 ./idt_tool -g
	taskset -c 0 ./idt_tool -c
	./idt_tool -z df
	./idt_tool -a ffffc900002ef000 -m addr:0xffffc90000317000 -v 8
	taskset -c 0 ./idt_tool -a ffffc900002ef000 -i


In [6]:
# An individual descriptor looks like this
run_cmd(symbios_path + "Tools/bin/idt_tool -p -v 14")

IDT at 0xffffc9000053f000
Descriptor 14 at 0xffffc9000053f0e0
about to load addr from desc
got idt addr
full addr: ffffc900004c7000
segment:   10
ist:       0
zero0:     0
type:      e
dpl:       0
p:         1


In [7]:
# A function that takes an output from the idt_tool, a string, and extracts the 
# corresponding value from the descriptor
def extract_desc_value(output, value):
    ret = output.decode("utf-8").split(value)[1].split("\n")[0]
    return hex(int(ret, 16))

In [8]:
import pandas as pd

def get_all_desc_values(df, output):
    addr = extract_desc_value(output, "full addr:")
    segment = extract_desc_value(output, "segment:")
    ist = extract_desc_value(output, "ist:")
    zero0 = extract_desc_value(output, "zero0:")
    type = extract_desc_value(output, "type:")
    dpl = extract_desc_value(output, "dpl:")
    p = extract_desc_value(output, "p:")
    # Add all varaiables as one row of a dataframe
    df.loc[len(df)] = [addr, segment, ist, zero0, type, dpl, p]

In [25]:
# Loop over all entries in the IDT adding to df
df = pd.DataFrame(columns=["addr", "segment", "ist", "zero0", "type", "dpl", "p"])

for i in range(0, 256):
    output = subprocess.check_output([symbios_path + "Tools/bin/idt_tool", "-p", "-v", str(i)])
    get_all_desc_values(df, output)
